## 安装与环境设置

In [1]:
!pip install langchain
!pip install openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain import FewShotPromptTemplate
from langchain.schema import(AIMessage,HumanMessage,SystemMessage)
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.9 MB/s eta 0:00:00


In [2]:
os.environ['OPENAI_API_KEY'] = 'sk-xxx'

## Few shot without question

In [5]:
def few_shot_cot_nqa(claim_text):
  examples = [
  {"claim": """ # The claim is that In 1959 , former Chilean boxer Alfredo Cornejo Cuevas (born June 6, 1933) won the gold medal in the welterweight division at the Pan American Games (held in Chicago, United States, from August 27 to September 7) in Chicago, United States, and the world amateur welterweight title in Mexico City. """,
  "program": """
  def program ():
    fact_1 = Verify (" Alfredo Cornejo Cuevas was born in June 6 , 1933. ")
    fact_2 = Verify (" Alfredo Cornejo Cuevas won the gold medal in the welterweight division at the Pan American Games in 1959. ")
    fact_3 = Verify (" The Pan American Games in 1959 was held in Chicago , United States , from August 27 to September 7.")
    fact_4 = Verify (" Alfredo Cornejo Cuevas won the world amateur welterweight title in Mexico City .")
    label = Predict ( fact_1 and fact_2 and fact_3 and fact_4 )
  """},

  {"claim": """ # The claim is that The Footwork FA12 , which was intended to start the season , finally debuted at the San Marino Grand Prix , a Formula One motor race held at Imola on 28 April 1991.""",
  "program": """
  def program ():
    fact_1 = Verify (" The Footwork FA12 , which was intended to start the season .")
    fact_2 = Verify (" The Footwork FA12 finally debuted at the San Marino Grand Prix .")
    fact_3 = Verify (" The San Marino Grand Prix was a Formula One motor race held at Imola on 28 April 1991. ")
    label = Predict ( fact_1 and fact_2 and fact_3 )
  """}]

  example_template = """claim: {claim}
program: {program}"""
  prefix = """Generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . """
  suffix ="""claim: {claim}"""
  example_prompt = PromptTemplate(input_variables = ["claim","program"],template=example_template)

  few_shot_prompt_template = FewShotPromptTemplate(
      examples = examples,
      example_prompt = example_prompt,
      prefix = prefix,
      suffix = suffix,
      input_variables = ["claim"],
      example_separator = "\n\n"
  )

  max_tokens_chat = 2000

  llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo',max_tokens=max_tokens_chat)
  from langchain.prompts.chat import (
      ChatPromptTemplate,
      SystemMessagePromptTemplate,
      HumanMessagePromptTemplate,
  )
  template = ''
  system_message_prompt = SystemMessagePromptTemplate.from_template(template)
  human_template="{text}"
  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
  chain = LLMChain(llm=llm, prompt=chat_prompt)
  answer = chain.run(text=few_shot_prompt_template.format(claim=claim_text))

  return answer,few_shot_prompt_template.format(claim=claim_text)

In [6]:
answer,input = few_shot_cot_nqa('# The claim is that Sumo wrestler Toyozakura Toshiaki committed match -fixing , ending his career in 2011 that started in 1989.')

In [7]:
print(input)
print(' ')
print(answer)

Generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . 

claim:  # The claim is that In 1959 , former Chilean boxer Alfredo Cornejo Cuevas (born June 6, 1933) won the gold medal in the welterweight division at the Pan American Games (held in Chicago, United States, from August 27 to September 7) in Chicago, United States, and the world amateur welterweight title in Mexico City. 
program: 
  def program ():
    fact_1 = Verify (" Alfredo Cornejo Cuevas was born in June 6 , 1933. ")
    fact_2 = Verify (" Alfredo Cornejo Cuevas won the gold medal in the welterweight division at the Pan American Games in 1959. ")
    fact_3 = Verify (" The Pan American Games in 1959 was held in Chicago , United States , from August 27 to September 7.")
    fact_4 = Verify (" Alfredo Co

## Few shot with question

In [14]:
def few_shot_cot_qa(claim_text):
  examples = [
  {"claim": """ # The country that Fujairah College is located in had a 2013 population of 9.2 million until it was hit by the plague in 1483 when the population was halved. """,
  "program": """
  def program ():
    answer_1 = Question("Which country is Fujairah College located in?")
    fact_1 = Verify("[answer_1] had a 2013 population of 9.2 million.")
    fact_2 = Verify("[answer_1] was hit by the plague in 1483.")
    fact_3 = Verify("The population of [answer_1] was halved in 1483.")
    label = Predict(fact_1 and fact_2 and fact_3)
  """},

  {"claim": """ # The solicitor who won the show Back to Reality ahead of Maureen Rees and Craig Phillips is English. The solicitor that was a chair of Global Witness is also English. """,
  "program": """
  def program ():
    answer_1 = Question("Which solicitor won the show Back to Reality ahead of Maureen Rees and Craig Phillips?")
    answer_2 = Question("Which solicitor was a chair of Global Witness?")
    fact_1 = Verify("[answer_1] is English.")
    fact_2 = Verify("[answer_2] is English.")
    label = Predict(fact_1 and fact_2)
  """}]

  example_template = """claim: {claim}
program: {program}"""
  prefix = """Generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . """
  suffix ="""claim: {claim}"""
  example_prompt = PromptTemplate(input_variables = ["claim","program"],template=example_template)

  few_shot_prompt_template = FewShotPromptTemplate(
      examples = examples,
      example_prompt = example_prompt,
      prefix = prefix,
      suffix = suffix,
      input_variables = ["claim"],
      example_separator = "\n\n"
  )

  max_tokens_chat = 2000

  llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo',max_tokens=max_tokens_chat)
  from langchain.prompts.chat import (
      ChatPromptTemplate,
      SystemMessagePromptTemplate,
      HumanMessagePromptTemplate,
  )
  template = ''
  system_message_prompt = SystemMessagePromptTemplate.from_template(template)
  human_template="{text}"
  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
  chain = LLMChain(llm=llm, prompt=chat_prompt)
  answer = chain.run(text=few_shot_prompt_template.format(claim=claim_text))

  return answer,few_shot_prompt_template.format(claim=claim_text)

In [15]:
answer,input = few_shot_cot_qa('# Anthony Burgess addressed the novelist and essayist, the author of Grimus, in a lengthy love letter. The author is of the same nationality as Raj Koothrappali.')

In [16]:
# inference error
print(input)
print(' ')
print(answer)

Generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . 

claim:  # The country that Fujairah College is located in had a 2013 population of 9.2 million until it was hit by the plague in 1483 when the population was halved. 
program: 
  def program ():
    answer_1 = Question("Which country is Fujairah College located in?")
    fact_1 = Verify("[answer_1] had a 2013 population of 9.2 million.")
    fact_2 = Verify("[answer_1] was hit by the plague in 1483.")
    fact_3 = Verify("The population of [answer_1] was halved in 1483.")
    label = Predict(fact_1 and fact_2 and fact_3)
  

claim:  # The solicitor who won the show Back to Reality ahead of Maureen Rees and Craig Phillips is English. The solicitor that was a chair of Global Witness is also English. 
program: 
 

In [17]:
# inference error
answer,input = few_shot_cot_qa('# The musician, who founded Morningwood with Max Green, is older than Max Green.')

print(input)
print(' ')
print(answer)

Generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . 

claim:  # The country that Fujairah College is located in had a 2013 population of 9.2 million until it was hit by the plague in 1483 when the population was halved. 
program: 
  def program ():
    answer_1 = Question("Which country is Fujairah College located in?")
    fact_1 = Verify("[answer_1] had a 2013 population of 9.2 million.")
    fact_2 = Verify("[answer_1] was hit by the plague in 1483.")
    fact_3 = Verify("The population of [answer_1] was halved in 1483.")
    label = Predict(fact_1 and fact_2 and fact_3)
  

claim:  # The solicitor who won the show Back to Reality ahead of Maureen Rees and Craig Phillips is English. The solicitor that was a chair of Global Witness is also English. 
program: 
 

In [18]:
# inference error
answer,input = few_shot_cot_qa('# The record producer that produced the bluegrass album was born on 22 June, 1944. This album inspired a Tony award winning musical. This musical had a character that was originated by Carmen Cusack.')

print(input)
print(' ')
print(answer)

Generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . 

claim:  # The country that Fujairah College is located in had a 2013 population of 9.2 million until it was hit by the plague in 1483 when the population was halved. 
program: 
  def program ():
    answer_1 = Question("Which country is Fujairah College located in?")
    fact_1 = Verify("[answer_1] had a 2013 population of 9.2 million.")
    fact_2 = Verify("[answer_1] was hit by the plague in 1483.")
    fact_3 = Verify("The population of [answer_1] was halved in 1483.")
    label = Predict(fact_1 and fact_2 and fact_3)
  

claim:  # The solicitor who won the show Back to Reality ahead of Maureen Rees and Craig Phillips is English. The solicitor that was a chair of Global Witness is also English. 
program: 
 

## Few shot with HOVER template

In [31]:
def few_shot_cot_HOVER(claim_text):
  examples = [
    {"claim": """ # The claim is that Howard University Hospital and Providence Hospital are both located in Washington, D.C. """,
    "program": """
    def program():
        fact_1 = Verify("Howard University Hospital is located in Washington, D.C.")
        fact_2 = Verify("Providence Hospital is located in Washington, D.C.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that WWE Super Tuesday took place at an arena that currently goes by the name TD Garden. """,
    "program": """
    def program():
        answer_1 = Question("Which arena the WWE Super Tuesday took place?")
        fact_1 = Verify(f"[answer_1] currently goes by the name TD Garden.")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Talking Heads, an American rock band that was "one of the most critically acclaimed bands of the 80's" is featured in KSPN's AAA format. """,
    "program": """
    def program():
        fact_1 = Verify("Talking Heads is an American rock band that was 'one of the most critically acclaimed bands of the 80's'.")
        fact_2 = Verify("Talking Heads is featured in KSPN's AAA format.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that An IndyCar race driver drove a Formula 1 car designed by Peter McCool during the 2007 Formula One season. """,
    "program": """
    def program():
        answer_1 = Question("Which Formula 1 car was designed by Peter McCool during the 2007 Formula One season?")
        fact_1 = Verify(f"An IndyCar race driver drove the car [answer_1].")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Gina Bramhill was born in a village. The 2011 population of the area that includes this village was 167,446. """,
    "program": """
    def program():
        answer_1 = Question("Which village was Gina Bramhill born in?")
        fact_1 = Verify(f"The 2011 population of the area that includes [answer_1] was 167,446.")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Don Ashley Turlington graduated from Saint Joseph's College, a private Catholic liberal arts college in Standish. """,
    "program": """
    def program():
        fact_1 = Verify("Saint Joseph's College is a private Catholic liberal arts college is located in Standish.")
        fact_2 = Verify(f"Don Ashley Turlington graduated from Saint Joseph's College.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that Gael and Fitness are not published in the same country. """,
    "program": """
    def program():
        answer_1 = Question("Which country was Gael published in?")
        answer_2 = Question("Which country was Fitness published in?")
        fact_1 = Verify(f"[answer_1] and [answer_2] are not the same country.")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Blackstar is the name of the album released by David Bowie that was recorded in secret. """,
    "program": """
    def program():
        fact_1 = Verify("David Bowie released an album called Blackstar.")
        fact_2 = Verify("David Bowie recorded an album in secret.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that In the 2004 Hockey film produced by a former major league baseball pitcher Kurt Russell played the USA coach. """,
    "program": """
    def program():
        answer_1 = Question("Which 2004 Hockey film was produced a former major league baseball pitcher?")
        fact_1 = Verify("Kurt Russell played the USA coach in the film [answer_1].")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Along with the New York Islanders and the New York Rangers, the New Jersey Devils NFL franchise is popular in the New York metropolitan area. """,
    "program": """
    def program():
        fact_1 = Verify("The New York Islanders and the New York Rangers are popular in the New York metropolitan area.")
        fact_2 = Verify("The New Jersey Devils NFL franchise is popular in the New York metropolitan area.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that Jack McFarland is the best known role of the host of the 64th Annual Tony Awards. """,
    "program": """
    def program():
        answer_1 = Question("Who is the host of the 64th Annual Tony Awards?")
        fact_1 = Verify(f\"Jack McFarland is the best known role of [answer_1].)
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that The song recorded by Fergie that was produced by Polow da Don and was followed by Life Goes On was M.I.L.F.$. """,
    "program": """
    def program():
        fact_1 = Verify("M.I.L.F.$ was recorded by Fergie that was produced by Polow da Don.")
        fact_2 = Verify("M.I.L.F.$ was was followed by Life Goes On.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that Eatza Pizza and Your Pie were not founded in the same state. """,
    "program": """
    def program():
        answer_1 = Question("Which state was Eatza Pizza founded in?")
        answer_2 = Question("Which state was Your Pie founded in?")
        fact_1 = Verify(f"[answer_1] and [answer_2] are not the same state.")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Gregg Rolie and Rob Tyner, are not a keyboardist. """,
    "program": """
    def program():
        fact_1 = Verify("Gregg Rolie is not a keyboardist.")
        fact_2 = Verify("Rob Tyner is not a keyboardist.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that Maria Esther Andion Bueno, not Jimmy Connors, is the player that is from Brazil. """,
    "program": """
    def program():
        fact_1 = Verify("Maria Esther Andion Bueno is from Brazil.")
        fact_2 = Verify("Jimmy Connors is not from Brazil.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that Vladimir Igorevich Arnold died after Georg Cantor. """,
    "program": """
    def program():
        answer_1 = Question("When did Vladimir Igorevich Arnold die?")
        answer_2 = Question("When did Georg Cantor die?")
        fact_1 = Verify(f"[answer_1] is after [answer_2].")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Barton Mine was halted by a natural disaster not Camlaren Mine. """,
    "program": """
    def program():
        fact_1 = Verify("Barton Mine was halted by a natural disaster.")
        fact_2 = Verify("Camlaren Mine was not halted by a natural disaster.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that John O'Hara and Rabindranath Tagore are not the same nationality. """,
    "program": """
    def program():
        answer_1 = Question("What is the nationality of John O'Hara?")
        answer_2 = Question("What is the nationality of Rabindranath Tagore?")
        fact_1 = Verify(f"[answer_1] and [answer_2] are not the same nationality.")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Thomas Loren Friedman has won more Pulitzer Prizes than Colson Whitehead. """,
    "program": """
    def program():
        answer_1 = Question("How many Pulitzer Prizes has Thomas Loren Friedman won?")
        answer_2 = Question("How many Pulitzer Prizes has Colson Whitehead won?")
        fact_1 = Verify(f"[answer_1] is more than [answer_2].")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that The model of car Trevor Bayne drives was introduced for model year 2006. The Rookie of The Year in the 1997 CART season drives it in the NASCAR Sprint Cup. """,
    "program": """
    def program():
        answer_1 = Question("Which model of car is drived by Trevor Bayne?")
        fact_1 = Verify(f"[answer_1] was introduced for model year 2006.")
        answer_2 = Question("Who is the Rookie of The Year in the 1997 CART season?")
        fact_2 = Verify(f"[answer_2] drives the model of car Trevor Bayne drives in the NASCAR Sprint Cup.")
        label = predict(fact_1 and fact_2)
    """},

]

  example_template = """claim: {claim}
program: {program}"""
  prefix = """Generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . """
  suffix ="""claim: {claim}"""
  example_prompt = PromptTemplate(input_variables = ["claim","program"],template=example_template)

  few_shot_prompt_template = FewShotPromptTemplate(
      examples = examples,
      example_prompt = example_prompt,
      prefix = prefix,
      suffix = suffix,
      input_variables = ["claim"],
      example_separator = "\n\n"
  )

  max_tokens_chat = 2000
  temperature_set = 0.7

  llm = ChatOpenAI(temperature=temperature_set,model_name='gpt-3.5-turbo',max_tokens=max_tokens_chat)
  from langchain.prompts.chat import (
      ChatPromptTemplate,
      SystemMessagePromptTemplate,
      HumanMessagePromptTemplate,
  )
  template = ''
  system_message_prompt = SystemMessagePromptTemplate.from_template(template)
  human_template="{text}"
  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
  chain = LLMChain(llm=llm, prompt=chat_prompt)
  answer = chain.run(text=few_shot_prompt_template.format(claim=claim_text))

  return answer,few_shot_prompt_template.format(claim=claim_text)

In [28]:
# inference error
answer,input = few_shot_cot_HOVER('# The musician, who founded Morningwood with Max Green, is older than Max Green.')

# print(input)
# print(' ')
print(answer)

program: 
    def program():
        answer_1 = Question("Who founded Morningwood with Max Green?")
        answer_2 = Question("Who is older, the musician or Max Green?")
        fact_1 = Verify(f"The musician who founded Morningwood with [answer_1] is older than Max Green.")
        label = Predict(fact_1)


## Few shot with FEVEROUS-S template

In [32]:
def few_shot_cot_FS(claim_text):
  examples = [
  {"claim": """ # The claim is that In 1959 , former Chilean boxer Alfredo Cornejo Cuevas (born June 6, 1933) won the gold medal in the welterweight division at the Pan American Games (held in Chicago, United States, from August 27 to September 7) in Chicago, United States, and the world amateur welterweight title in Mexico City. """,
  "program": """
  def program ():
    fact_1 = Verify (" Alfredo Cornejo Cuevas was born in June 6 , 1933. ")
    fact_2 = Verify (" Alfredo Cornejo Cuevas won the gold medal in the welterweight division at the Pan American Games in 1959. ")
    fact_3 = Verify (" The Pan American Games in 1959 was held in Chicago , United States , from August 27 to September 7.")
    fact_4 = Verify (" Alfredo Cornejo Cuevas won the world amateur welterweight title in Mexico City .")
    label = Predict ( fact_1 and fact_2 and fact_3 and fact_4 )
  """},

  {"claim": """ # The claim is that The Footwork FA12 , which was intended to start the season , finally debuted at the San Marino Grand Prix , a Formula One motor race held at Imola on 28 April 1991.
  """,
  "program": """
  def program ():
    fact_1 = Verify (" The Footwork FA12 , which was intended to start the season .")
    fact_2 = Verify (" The Footwork FA12 finally debuted at the San Marino Grand Prix .")
    fact_3 = Verify (" The San Marino Grand Prix was a Formula One motor race held at Imola on 28 April 1991. ")
    label = Predict ( fact_1 and fact_2 and fact_3 )
  """},

    {"claim": """ # The claim is that SkyHigh Mount Dandenong (formerly Mount Dandenong Observatory) is a restaurant located on top of Mount Dandenong, Victoria, Australia. """,
  "program": """
def program():
    fact_1 = Verify("SkyHigh Mount Dandenong is a restaurant located on top of Mount Dandenong, Victoria, Australia.")
    fact_2 = Verify("SkyHigh Mount Dandenong is formerly known as Mount Dandenong Observatory.")
    label = Predict(fact_1 and fact_2)
  """},

      {"claim": """# The claim is that Before the first Europeans arrived or copra companies leased it, Maupihaa was home to Inca's in ancient times. """,
  "program": """
def program():
    fact_1 = Verify("Maupihaa was home to Inca's in ancient times.")
    fact_2 = Verify("Maupihaa was home to Inca's before the first Europeans arrived or copra companies leased it.")
    label = Predict(fact_1 and fact_2)
  """},

      {"claim": """# The claim is that Shulin, a 33.1288 km (12.7911 sq mi) land located in New Taipei City, China, a country in East Asia, has a total population of 183,946 in December 2018.""",
  "program": """
def program():
    fact_1 = Verify("Shulin is a 33.1288 km (12.7911 sq mi) land located in New Taipei City, China.")
    fact_2 = Verify("Shulin has a total population of 183,946 in December 2018.")
    label = Predict(fact_1 and fact_2)
  """},

      {"claim": """
      # The claim is that Sumo wrestler Toyozakura Toshiaki committed match-fixing, ending his career in 2011 that started in 1989.
      """,
  "program": """
def program():
    fact_1 = Verify("Toyozakura Toshiaki ended his career in 2011 that started in 1989.")
    fact_2 = Verify("Toyozakura Toshiaki is a Sumo wrestler.")
    fact_3 = Verify("Toyozakura Toshiaki committed match-fixing.")
    label = Predict(fact_1 and fact_2 and fact_3)
  """},

        {"claim": """
      # The claim is that In 1959, former Chilean boxer Alfredo Cornejo Cuevas (born June 6, 1933) won the gold medal in the welterweight division at the Pan American Games (held in Chicago, United States, from August 27 to September 7) in Chicago, United States, and the world amateur welterweight title in Mexico City.
      """,
  "program": """
def program():
    fact_1 = Verify("Alfredo Cornejo Cuevas is a former Chilean boxer.")
    fact_2 = Verify("Alfredo Cornejo won the gold medal in the welterweight division at the Pan American Games.")
    fact_3 = Verify("The Pan American Games was held in Chicago, United States, from August 27 to September 7.")
    fact_4 = Verify("Alfredo Cornejo won the world amateur welterweight title in Mexico City.")
    label = Predict(fact_1 and fact_2 and fact_3 and fact_4)
  """},
        {"claim": """
      # The claim is that Adductor hiatus is associated with nine structures, seven of which enter and leave through hiatus.
      """,
  "program": """
def program():
    fact_1 = Verify("Adductor hiatus is associated with nine structures.")
    fact_2 = Verify("Seven of the nine structures associated with Adductor hiatus enter and leave through hiatus.")
    label = Predict(fact_1 and fact_2)
  """},
          {"claim": """
      # The claim is that Ifor Bowen Lloyd was educated at Winchester (an independent boarding school for boys in the British public school tradition) and Exeter College, Oxford where he was a member of the Library Committee of the Oxford Union Society, as well as, received a BA in Modern History in 1924.
      """,
  "program": """
def program():
    fact_1 = Verify("Ifor Bowen Lloyd was educated at Winchester and Exeter College, Oxford.")
    fact_2 = Verify("Winchester is an independent boarding school for boys in the British public school tradition.")
    fact_3 = Verify("While at Oxford, Ifor Bowen Lloyd was a member of the Library Committee of the Oxford Union Society.")
    fact_4 = Verify("Ifor Bowen Lloyd received a BA in Modern History in 1924 at Oxford.")
    label = Predict(fact_1 and fact_2 and fact_3 and fact_4)
  """},
          {"claim": """
      # The claim is that In the 2001 Stanley Cup playoffs Eastern Conference Semifinals Devils' Elias scored and Maple Leafs' left Devils player Scott Neidermayer hurt.
      """,
  "program": """
def program():
    fact_1 = Verify("In the 2001 Stanley Cup playoffs Eastern Conference Semifinals Devils' Elias scored.")
    fact_2 = Verify("Maple Leafs' left Devils player Scott Neidermayer hurt.")
    label = Predict(fact_1 and fact_2)
  """},
          {"claim": """
      # The claim is that Teldenia helena is a moth first described in 1967 by Wilkinson.
      """,
  "program": """
def program():
    fact_1 = Verify("Teldenia helena is a moth.")
    fact_2 = Verify("Teldenia helena was first described by Wilkinson in 1967.")
    label = Predict(fact_1 and fact_2)
  """},
          {"claim": """
      # The claim is that Born December 30, 1974, William Frick was a dark horse candidate in the Maryland House of Delegates appointment process.
      """,
  "program": """
def program():
    fact_1 = Verify("William Frick was born in December 30, 1974.")
    fact_2 = Verify("William Frick was a dark horse candidate in the Maryland House of Delegates appointment process.")
    label = Predict(fact_1 and fact_2)
  """}
  ]

  example_template = """claim: {claim}
program: {program}"""
  prefix = """Generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . """
  suffix ="""claim: {claim}"""
  example_prompt = PromptTemplate(input_variables = ["claim","program"],template=example_template)

  few_shot_prompt_template = FewShotPromptTemplate(
      examples = examples,
      example_prompt = example_prompt,
      prefix = prefix,
      suffix = suffix,
      input_variables = ["claim"],
      example_separator = "\n\n"
  )

  max_tokens_chat = 2000
  temperature_set = 0.7

  llm = ChatOpenAI(temperature=temperature_set,model_name='gpt-3.5-turbo',max_tokens=max_tokens_chat)
  from langchain.prompts.chat import (
      ChatPromptTemplate,
      SystemMessagePromptTemplate,
      HumanMessagePromptTemplate,
  )
  template = ''
  system_message_prompt = SystemMessagePromptTemplate.from_template(template)
  human_template="{text}"
  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
  chain = LLMChain(llm=llm, prompt=chat_prompt)
  answer = chain.run(text=few_shot_prompt_template.format(claim=claim_text))

  return answer,few_shot_prompt_template.format(claim=claim_text)

In [23]:
# inference error
answer,input = few_shot_cot_FS('# The musician, who founded Morningwood with Max Green, is older than Max Green.')

# print(input)
# print(' ')
print(answer)

program: 
def program():
    fact_1 = Verify("The musician founded Morningwood with Max Green.")
    fact_2 = Verify("The musician is older than Max Green.")
    label = Predict(fact_1 and fact_2)


## example from paper

In [38]:
sen = '# The critically acclaimed film, that Buddy Baker scored in 1975, is a Walt Disney film. It was produced first before the film that featured Bruce M. Fischer as Mr. Coogar. '

# HOVER
answer_hr,input = few_shot_cot_HOVER(sen)
# FEVEROUS-S
answer_fs,input = few_shot_cot_FS(sen)

print(answer_hr)
print('\n')
print(answer_fs)

program: 
    def program():
        answer_1 = Question("Which critically acclaimed film did Buddy Baker score in 1975?")
        fact_1 = Verify(f"The film [answer_1] is a Walt Disney film.")
        answer_2 = Question("Which film featured Bruce M. Fischer as Mr. Coogar?")
        fact_2 = Verify(f"The film produced first before [answer_2] is the film Buddy Baker scored in 1975.")
        label = Predict(fact_1 and fact_2)


program: 
def program():
    fact_1 = Verify("The critically acclaimed film was scored by Buddy Baker in 1975.")
    fact_2 = Verify("The critically acclaimed film is a Walt Disney film.")
    fact_3 = Verify("The film featuring Bruce M. Fischer as Mr. Coogar was produced after the critically acclaimed film.")
    label = Predict(fact_1 and fact_2 and fact_3)
